In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)
# quiet_library(ArchR)
quiet_library(cowplot)

In [ ]:
mnp <- readRDS(file = 'mnp_deep_analysis.rds')

In [ ]:
mnp <- FindClusters(mnp, graph.name = "wsnn", algorithm = 3, resolution = 0.8, verbose = TRUE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 8)
p1 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.5', label = TRUE)
p2 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', shuffle = TRUE)
p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 8)
p1 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.5', label = TRUE)
p2 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'CMV', shuffle = TRUE)
p1 + p2

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 8)
p1 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.8', label = TRUE, label.size = 10)
p2 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', shuffle = TRUE)
p1 + p2

In [ ]:
mnp <- SetIdent(mnp, value = 'wsnn_res.0.8')
mnp <- FindSubCluster(object = mnp, cluster = 3, graph.name = 'wsnn', resolution = 0.5)
Idents(mnp) <- "sub.cluster"

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 8)
p1 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'sub.cluster', label = TRUE, label.size = 10)
p2 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'CMV', shuffle = TRUE)
p1 + p2

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'sub.cluster', label = TRUE, label.size = 10) & NoLegend()

# Fig6H

In [ ]:
mnp <- SetIdent(mnp, value = 'sub.cluster')
mnp <- RenameIdents(mnp,
                    '0' = '1',
                    '1' = '2',
                    '2' = '3',
                    '3_0' = '4',
                    '3_1' = '5',
                    '4' = '6',
                    '5' = '7')
mnp$sub.cluster.clean <- Idents(mnp)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'sub.cluster.clean', label = TRUE, label.size = 10) & NoLegend()

In [ ]:
# Open a pdf file
pdf("plots/mnp2_subcluster_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'sub.cluster.clean', label = FALSE) & NoLegend()
# Close the pdf file
dev.off() 

In [ ]:
mnp_split <- SplitObject(mnp, split.by = 'pediatric_senior')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
DimPlot(mnp_split$Pediatric, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', cols = c('#1b9e77')) & NoLegend()
DimPlot(mnp_split$Senior, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', cols = c('#d95f02')) & NoLegend()

In [ ]:
# Open a pdf file
pdf("plots/mnp2_subcluster_pediatric_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
DimPlot(mnp_split$Pediatric, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', cols = c('#1b9e77')) & NoLegend()
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_subcluster_senior_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
DimPlot(mnp_split$Senior, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', cols = c('#d95f02')) & NoLegend()
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 8)
DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', label = FALSE, split.by = 'pediatric_senior', cols = c('#d95f02','#1b9e77'))

In [ ]:
mnp_split <- SplitObject(mnp, split.by = 'pediatric_senior')

In [ ]:
wnn_df <- as.data.frame(mnp@reductions$wnn.3.umap@cell.embeddings)
wnn_df$age <- mnp$pediatric_senior
wnn_df$cmv <- mnp$CMV
head(wnn_df)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
wnn_df %>%
    mutate(age = factor(age, levels = c('Senior','Pediatric'))) %>%
    ggplot(aes(x = wnn3umap_1, y = wnn3umap_2, color = age)) + #geom_point(shape = ".") +
    stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = age), bins = 10) + xlim(-6,7) + ylim(-6,6) + scale_fill_manual(values = c('#d95f02','#1b9e77')) + scale_color_manual(values = c('#d95f02','#1b9e77')) +
    theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank(),axis.line = element_line(colour = "black"),
          legend.position = 'none') 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
ggplot(wnn_df, aes(x = wnn3umap_1, y = wnn3umap_2, color = cmv)) + #geom_point(shape = ".") +
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = cmv), bins = 10) + xlim(-6,7) + ylim(-6,6) + scale_fill_manual(values = c('#d8b365','#5ab4ac')) + scale_color_manual(values = c('#d8b365','#5ab4ac')) +
theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), panel.background = element_blank(),axis.line = element_line(colour = "black"),
      legend.position = 'none')

# Age Distribution

In [ ]:
cluster_counts <- dplyr::count(mnp@meta.data, sub.cluster, pbmc_sample_id, pediatric_senior)

In [ ]:
sum_counts <- mnp@meta.data %>%
                dplyr::count(pediatric_senior, pbmc_sample_id)

In [ ]:
joined_counts <- left_join(cluster_counts, sum_counts, by = 'pbmc_sample_id')
joined_counts$perc <- joined_counts$n.x/joined_counts$n.y

In [ ]:
head(joined_counts)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
ggplot(joined_counts, aes(x = sub.cluster, y = perc, fill = pediatric_senior.x)) + 
    geom_boxplot() + 
    scale_y_log10() + 
    geom_dotplot(binaxis='y', stackdir='center',
                 position=position_dodge(), binwidth = 1/40) + theme_bw() +
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5, size = 15),
          axis.text.y = element_text(size = 15)) +
    guides(fill = guide_legend(title = 'Pediatric / Senior')) +
    xlab("Cluster") + 
    ylab("Percent")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
ggplot(joined_counts, aes(x = sub.cluster, y = perc, fill = pediatric_senior.x)) + 
    geom_boxplot() + 
    geom_dotplot(binaxis='y', stackdir='center',
                 position=position_dodge(), binwidth = 1/40) + theme_bw() +
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5, size = 15),
          axis.text.y = element_text(size = 15)) +
    guides(fill = guide_legend(title = 'Pediatric / Senior')) +
    xlab("Cluster") + 
    ylab("Percent")

In [ ]:
subset(joined_counts, sub.cluster == 5)

In [ ]:
mnp <- SetIdent(mnp, value = 'sub.cluster')
mnp_pure <- subset(mnp, idents = c('3_1','5'), invert = TRUE)

In [ ]:
celltype_df_func <- function(seurat_object, metadata_slot, id_list){
    seurat_object <- SetIdent(seurat_object, value = metadata_slot)
    subset_object <- subset(seurat_object, idents = id_list[[1]])
    df <- as.data.frame(table(subset_object$pbmc_sample_id))
    colnames(df) <- c('sample',id_list[[1]])
    i = 2
    while(i < length(id_list) + 1){
        subset_object <- subset(seurat_object, idents = id_list[[i]])
        df2 <- as.data.frame(table(subset_object$pbmc_sample_id))
        df[,ncol(df) + 1] <- df2$Freq
        colnames(df)[ncol(df)] <- paste0(id_list[[i]])
        #df$id_list[[i]] <- df2$Freq
        i = i + 1
        }
    df
    }

In [ ]:
table(mnp_pure$pbmc_sample_id)

In [ ]:
cluster_counts <- dplyr::count(mnp_pure@meta.data, sub.cluster, pbmc_sample_id, pediatric_senior)

In [ ]:
sum_counts <- mnp_pure@meta.data %>%
                dplyr::count(pediatric_senior, pbmc_sample_id)

In [ ]:
joined_counts <- left_join(cluster_counts, sum_counts, by = 'pbmc_sample_id')
joined_counts$perc <- joined_counts$n.x/joined_counts$n.y

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
ggplot(joined_counts, aes(x = sub.cluster, y = perc, fill = pediatric_senior.x)) + 
    geom_boxplot() + 
    geom_dotplot(binaxis='y', stackdir='center',
                 position=position_dodge(), binwidth = 1/40) + theme_bw() +
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5, size = 15),
          axis.text.y = element_text(size = 15)) +
    guides(fill = guide_legend(title = 'Pediatric / Senior')) +
    xlab("Cluster") + 
    ylab("Percent")

In [ ]:
cluster_counts <- dplyr::count(mnp@meta.data, sub.cluster, pediatric_senior)
sum_counts <- mnp@meta.data %>%
                dplyr::count(pediatric_senior)

In [ ]:
joined_counts <- left_join(cluster_counts, sum_counts, by = 'pediatric_senior')
joined_counts$perc <- joined_counts$n.x/joined_counts$n.y

In [ ]:
joined_counts

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 10)
ggplot(joined_counts, aes(x = pediatric_senior, y = perc, fill = sub.cluster)) + 
    geom_bar(position="fill", stat="identity") + theme_bw()

In [ ]:
cluster_counts <- dplyr::count(mnp_pure@meta.data, sub.cluster, pediatric_senior)
sum_counts <- mnp_pure@meta.data %>%
                dplyr::count(pediatric_senior)

In [ ]:
joined_counts <- left_join(cluster_counts, sum_counts, by = 'pediatric_senior')
joined_counts$perc <- joined_counts$n.x/joined_counts$n.y

In [ ]:
joined_counts

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
ggplot(joined_counts, aes(x = pediatric_senior, y = perc, fill = sub.cluster)) + 
    geom_bar(position="fill", stat="identity") + theme_bw()

In [ ]:
mnp_age <- SplitObject(mnp, split.by = 'pediatric_senior')
mnp_cmv <- SplitObject(mnp_age$Pediatric, split.by = 'CMV')

## ExtFig8d

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5)
DimPlot(mnp_cmv$Negative, group.by = 'sub.cluster', split.by = 'pbmc_sample_id', ncol = 4, reduction = 'wnn.3.umap', pt.size = 1) & NoLegend()
DimPlot(mnp_cmv$Positive, group.by = 'sub.cluster', split.by = 'pbmc_sample_id', ncol = 4, reduction = 'wnn.3.umap', pt.size = 1) & NoLegend()

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pediatric_cmv_neg.pdf", width = 20, height = 5) 
# 2. Create a plot
DimPlot(mnp_cmv$Negative, group.by = 'sub.cluster', split.by = 'pbmc_sample_id', ncol = 4, reduction = 'wnn.3.umap', pt.size = 1.5) & NoLegend()
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pediatric_cmv_pos.pdf", width = 20, height = 5) 
# 2. Create a plot
DimPlot(mnp_cmv$Positive, group.by = 'sub.cluster', split.by = 'pbmc_sample_id', ncol = 4, reduction = 'wnn.3.umap', pt.size = 1.5) & NoLegend()
# Close the pdf file
dev.off() 

In [ ]:
celltype_df_func <- function(seurat_object, metadata_slot, id_list){
    seurat_object <- SetIdent(seurat_object, value = metadata_slot)
    subset_object <- subset(seurat_object, idents = id_list[[1]])
    df <- as.data.frame(table(subset_object$pbmc_sample_id))
    colnames(df) <- c('sample',id_list[[1]])
    i = 2
    while(i < length(id_list) + 1){
        subset_object <- subset(seurat_object, idents = id_list[[i]])
        df2 <- as.data.frame(table(subset_object$pbmc_sample_id))
        df[,ncol(df) + 1] <- df2$Freq
        colnames(df)[ncol(df)] <- paste0(id_list[[i]])
        #df$id_list[[i]] <- df2$Freq
        i = i + 1
        }
    df
    }

In [ ]:
ped_negative <- celltype_df_func(mnp_cmv$Negative, 'sub.cluster.clean', sort(unique(mnp_cmv$Negative$sub.cluster.clean)))

In [ ]:
head(ped_negative)

In [ ]:
rownames(ped_negative) <- ped_negative$sample
ped_negative <- ped_negative[,2:8]
colnames(ped_negative) <- sort(unique(mnp_cmv$Negative$sub.cluster.clean))
ped_negative

In [ ]:
ped_sums_neg <- rowSums(ped_negative)

In [ ]:
ped_perc_neg <- ped_negative / ped_sums_neg
ped_perc_neg

In [ ]:
ped_perc_neg$sample <- rownames(ped_perc_neg)
head(ped_perc_neg)

In [ ]:
# col_order <- c('CD4 Naive','CD4 CM','CD4 EM1','CD4 EM2','CD4 TEMRA','Treg','CD8 Naive','CD8 CM','CD8 EM1','CD8 EM2','CD8 TEMRA','sample')
# ped_perc2 <- ped_perc[,col_order]
# head(ped_perc2)

In [ ]:
library(tidyr)

In [ ]:
ped_percs_neg_tall <- gather(data = ped_perc_neg, key = cluster, value = cluster_perc, `1`:`7`, factor_key = TRUE)
head(ped_percs_neg_tall)

### Plot - ExtFig 8e

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 12)
ggplot(ped_percs_neg_tall, aes(x = sample, y = cluster_perc, fill = cluster, label = round(cluster_perc,2))) + 
       geom_bar(stat = "identity", position = 'fill', colour = 'white') + 
       # scale_fill_manual(values = c('#64B200','#F8766D','#DB8E00','#AEA200','#00BD5C','#FF63B6','#B385FF','#00C1A7','#00BADE','#00A6FF','#EF67EB')) +
       theme(axis.text.x = element_text(size = 15, angle = 90, hjust = 1, vjust = 0.5), axis.title.x = element_text(size = 20))

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pediatric_cmv_neg_barplot.pdf", width = 4, height = 12) 
# 2. Create a plot
ggplot(ped_percs_neg_tall, aes(x = sample, y = cluster_perc, fill = cluster, label = round(cluster_perc,2))) + 
       geom_bar(stat = "identity", position = 'fill', colour = 'white') + 
       # scale_fill_manual(values = c('#64B200','#F8766D','#DB8E00','#AEA200','#00BD5C','#FF63B6','#B385FF','#00C1A7','#00BADE','#00A6FF','#EF67EB')) +
       theme(axis.text.x = element_text(size = 15, angle = 90, hjust = 1, vjust = 0.5), axis.title.x = element_text(size = 20))
# Close the pdf file
dev.off() 

In [ ]:
ped_positive <- celltype_df_func(mnp_cmv$Positive, 'sub.cluster.clean', sort(unique(mnp_cmv$Positive$sub.cluster.clean)))

In [ ]:
head(ped_positive)

In [ ]:
rownames(ped_positive) <- ped_positive$sample
ped_positive <- ped_positive[,2:8]
colnames(ped_positive) <- sort(unique(mnp_cmv$Positive$sub.cluster.clean))
ped_positive

In [ ]:
ped_sums_pos <- rowSums(ped_positive)

In [ ]:
ped_perc_pos <- ped_positive / ped_sums_pos
ped_perc_pos

In [ ]:
ped_perc_pos$sample <- rownames(ped_perc_pos)
head(ped_perc_pos)

In [ ]:
# col_order <- c('CD4 Naive','CD4 CM','CD4 EM1','CD4 EM2','CD4 TEMRA','Treg','CD8 Naive','CD8 CM','CD8 EM1','CD8 EM2','CD8 TEMRA','sample')
# ped_perc2 <- ped_perc[,col_order]
# head(ped_perc2)

In [ ]:
library(tidyr)

In [ ]:
ped_percs_pos_tall <- gather(data = ped_perc_pos, key = cluster, value = cluster_perc, `1`:`7`, factor_key = TRUE)
head(ped_percs_pos_tall)

### Plot

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 12)
ggplot(ped_percs_pos_tall, aes(x = sample, y = cluster_perc, fill = cluster, label = round(cluster_perc,2))) + 
       geom_bar(stat = "identity", position = 'fill', colour = 'white') + 
       # scale_fill_manual(values = c('#64B200','#F8766D','#DB8E00','#AEA200','#00BD5C','#FF63B6','#B385FF','#00C1A7','#00BADE','#00A6FF','#EF67EB')) +
       theme(axis.text.x = element_text(size = 15, angle = 90, hjust = 1, vjust = 0.5), axis.title.x = element_text(size = 20))

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pediatric_cmv_pos_barplot.pdf", width = 4, height = 12) 
# 2. Create a plot
ggplot(ped_percs_pos_tall, aes(x = sample, y = cluster_perc, fill = cluster, label = round(cluster_perc,2))) + 
       geom_bar(stat = "identity", position = 'fill', colour = 'white') + 
       # scale_fill_manual(values = c('#64B200','#F8766D','#DB8E00','#AEA200','#00BD5C','#FF63B6','#B385FF','#00C1A7','#00BADE','#00A6FF','#EF67EB')) +
       theme(axis.text.x = element_text(size = 15, angle = 90, hjust = 1, vjust = 0.5), axis.title.x = element_text(size = 20))
# Close the pdf file
dev.off() 

In [ ]:
write.csv(ped_perc_neg, 'mnp2_ped_perc_neg.csv')
write.csv(ped_perc_pos, 'mnp2_ped_perc_pos.csv')

In [ ]:
ped_sums_neg
ped_sums_pos

# Pediatric BarPlots

In [ ]:
celltype_df_func <- function(seurat_object, metadata_slot, id_list){
    seurat_object <- SetIdent(seurat_object, value = metadata_slot)
    subset_object <- subset(seurat_object, idents = id_list[[1]])
    df <- as.data.frame(table(subset_object$pbmc_sample_id))
    colnames(df) <- c('sample',id_list[[1]])
    i = 2
    while(i < length(id_list) + 1){
        subset_object <- subset(seurat_object, idents = id_list[[i]])
        df2 <- as.data.frame(table(subset_object$pbmc_sample_id))
        df[,ncol(df) + 1] <- df2$Freq
        colnames(df)[ncol(df)] <- paste0(id_list[[i]])
        #df$id_list[[i]] <- df2$Freq
        i = i + 1
        }
    df
    }

In [ ]:
mnp_split <- SplitObject(mnp, split.by = 'pediatric_senior')

In [ ]:
ped_celltype <- celltype_df_func(mnp_split$Pediatric, 'sub.cluster', sort(unique(mnp_split$Pediatric$sub.cluster)))

In [ ]:
head(ped_celltype)

In [ ]:
rownames(ped_celltype) <- ped_celltype$sample
ped_celltype <- ped_celltype[,2:8]
colnames(ped_celltype) <- sort(unique(mnp_split$Pediatric$sub.cluster))
ped_celltype

In [ ]:
ped_sums <- rowSums(ped_celltype)

In [ ]:
ped_perc <- ped_celltype / ped_sums
ped_perc

In [ ]:
ped_perc$sample <- rownames(ped_perc)
head(ped_perc)

In [ ]:
# col_order <- c('CD4 Naive','CD4 CM','CD4 EM1','CD4 EM2','CD4 TEMRA','Treg','CD8 Naive','CD8 CM','CD8 EM1','CD8 EM2','CD8 TEMRA','sample')
# ped_perc2 <- ped_perc[,col_order]
# head(ped_perc2)

In [ ]:
library(tidyr)

In [ ]:
ped_percs_tall <- gather(data = ped_perc, key = cluster, value = cluster_perc, `0`:`5`, factor_key = TRUE)
head(ped_percs_tall)

### Plot

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 12)
ggplot(ped_percs_tall, aes(x = sample, y = cluster_perc, fill = cluster, label = round(cluster_perc,2))) + 
       geom_bar(stat = "identity", position = 'fill', colour = 'white') + 
       # scale_fill_manual(values = c('#64B200','#F8766D','#DB8E00','#AEA200','#00BD5C','#FF63B6','#B385FF','#00C1A7','#00BADE','#00A6FF','#EF67EB')) +
       theme(axis.text.x = element_text(size = 15, angle = 90, hjust = 1, vjust = 0.5), axis.title.x = element_text(size = 20))

# Cluster DEGs

In [ ]:
DefaultAssay(mnp) <- 'RNA'
mnp <- NormalizeData(mnp, assay = 'RNA') %>% ScaleData(features = rownames(mnp[['RNA']]))

In [ ]:
cluster_degs <- FindAllMarkers(mnp, assay = 'RNA', group.by = 'sub.cluster.clean')

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)
cluster_degs %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10
DoHeatmap(mnp, features = top10$gene) + NoLegend()

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)
FeaturePlot(mnp, features = c('KLRC2','GZMK','MME','GNG4','IKZF2','ZNF683'), ncol = 3, reduction = 'wnn.3.umap', order = TRUE)

# Fig6k

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 10)
FeaturePlot(mnp, features = c('LEF1','TBX21'), ncol = 1, reduction = 'wnn.3.umap', order = TRUE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)
FeaturePlot(mnp, features = c('KLRC2','KLRC3','MME','GNG4','IKZF2','ZNF683'), ncol = 3, reduction = 'wnn.3.umap', order = TRUE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)
FeaturePlot(mnp, features = c('GNLY','TBX21','ITGB1','CXCR4','TOX','CR1'), ncol = 3, reduction = 'wnn.3.umap', order = TRUE)

In [ ]:
FeaturePlot(mnp, features = c('TRAC','TRBC1','TRBC2','TRDC','TRGC1','TRGC2'), ncol = 3, reduction = 'wnn.3.umap', order = TRUE)

# ADT Expression

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 7)
FeaturePlot(mnp, features = c('cleanadt_CD244','cleanadt_CD11b'), ncol = 3, reduction = 'wnn.3.umap')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
plot_density(mnp, features = c('cleanadt_CD244','cleanadt_CD11b'), joint = TRUE)

In [ ]:
table(mnp$pbmc_sample_id)

In [ ]:
saveRDS(mnp, file = 'mnp_deep_analysis_2.rds')

# Full Expression - Fig6j

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
DotPlot(mnp, col.min = 0, dot.scale = 8, features = c('KLRC2','ZNF683','MME','GZMK','CR1','TBX21',
                          'GNG4','IKZF2','SOX4','STAT4','IL21R','PDCD1',
                          'GNLY','adt_CD279','adt_TIGIT','adt_CD4','adt_CD8a','adt_CD244','adt_CD11b'), group.by = 'sub.cluster.clean') & 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5),
      axis.text.y = element_text(angle = 90, hjust = 0.5, vjust = 1))

In [ ]:
# Open a pdf file
pdf("plots/mnp2_dotplot.pdf", width = 8, height = 4) 
# 2. Create a plot
DotPlot(mnp, col.min = 0, dot.scale = 8, features = c('KLRC2','ZNF683','MME','GZMK','CR1','TBX21',
                          'GNG4','IKZF2','SOX4','STAT4','IL21R','PDCD1',
                          'GNLY','adt_CD279','adt_TIGIT','adt_CD4','adt_CD8a','adt_CD244','adt_CD11b'), group.by = 'sub.cluster.clean') & 
theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5),
      axis.text.y = element_text(angle = 90, hjust = 0.5, vjust = 1))
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
FeaturePlot(mnp, features = c('cleanadt_CD45RA','cleanadt_CD197','SELL','cleanadt_CD28','cleanadt_CD27'), ncol = 3, reduction = 'wnn.3.umap')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'LEF1', reduction = 'wnn.3.umap')
FeaturePlot(mnp, features = 'TBX21', reduction = 'wnn.3.umap')

In [ ]:
# Open a pdf file
pdf("plots/mnp2_lef1_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'LEF1', reduction = 'wnn.3.umap')
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_tbx21_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'TBX21', reduction = 'wnn.3.umap')
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'KLRC2', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
FeaturePlot(mnp, features = 'IL21R', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'ZBTB16', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
FeaturePlot(mnp, features = 'IL18R1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'adt_CD11b', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')
FeaturePlot(mnp, features = 'adt_CD244', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')

In [ ]:
# Open a pdf file
pdf("plots/mnp2_ZBTB16_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'ZBTB16', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_IL18R1_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'IL18R1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_CD244_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'adt_CD244', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_CD11b_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'adt_CD11b', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_KLRC2_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'KLRC2', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
# Close the pdf file
dev.off() 

In [ ]:
# Open a pdf file
pdf("plots/mnp2_IL21R_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'IL21R', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'MME', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
FeaturePlot(mnp, features = 'CR1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'PDCD1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
FeaturePlot(mnp, features = 'adt_CD279', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'adt_CD4', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')
FeaturePlot(mnp, features = 'adt_CD8a', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 16)
p1 <- FeaturePlot(mnp, features = 'MME', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
p2 <- FeaturePlot(mnp, features = 'CR1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
p3 <- FeaturePlot(mnp, features = 'PDCD1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
p4 <- FeaturePlot(mnp, features = 'adt_CD279', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')
p5 <- FeaturePlot(mnp, features = 'adt_CD4', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')
p6 <- FeaturePlot(mnp, features = 'adt_CD8a', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis(option = 'inferno')

In [ ]:
plot_grid(p1, p2, p5, p3, p4, p6, ncol = 3)

In [ ]:
# Open a pdf file
pdf("plots/mnp2_extfig8_b_umap.pdf", width = 24, height = 16) 
# 2. Create a plot
plot_grid(p1, p2, p5, p3, p4, p6, ncol = 3)
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 16)
p1 <- FeaturePlot(mnp, features = 'GZMK', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
p2 <- FeaturePlot(mnp, features = 'GNLY', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
p3 <- FeaturePlot(mnp, features = 'ITGB1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
plot_grid(p1, p2, p3, ncol = 2)

In [ ]:
# Open a pdf file
pdf("plots/mnp2_extfig8_c_umap.pdf", width = 16, height = 16) 
# 2. Create a plot
plot_grid(p1, p2, p3, ncol = 2)
# Close the pdf file
dev.off()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeaturePlot(mnp, features = 'FOXP1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()

In [ ]:
# Open a pdf file
pdf("plots/mnp2_FOXP1_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
FeaturePlot(mnp, features = 'FOXP1', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1.5) & scale_color_viridis()
# Close the pdf file
dev.off()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
DimPlot(mnp, group.by = 'sub.cluster.clean', reduction = 'wnn.3.umap')

In [ ]:
mnp <- SetIdent(mnp, value = 'sub.cluster.clean')
mnp <- RenameIdents(mnp,
                    '1' = '1',
                    '2' = '1',
                    '3' = '1',
                    '4' = '2',
                    '5' = '3',
                    '6' = '4',
                    '7' = '5')
mnp$condensed.clusters <- Idents(mnp)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
DimPlot(mnp, group.by = 'condensed.clusters', reduction = 'wnn.3.umap', pt.size = 1.5) & NoLegend()

In [ ]:
# Open a pdf file
pdf("plots/mnp2_condensed_clusters_umap.pdf", width = 8, height = 8) 
# 2. Create a plot
DimPlot(mnp, group.by = 'condensed.clusters', reduction = 'wnn.3.umap', pt.size = 1.5) & NoLegend()
# Close the pdf file
dev.off()